In [21]:
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, confusion_matrix, accuracy_score, recall_score
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, LabelEncoder
import datetime
import os
import json

In [22]:
# Parámetros
inputFolder = "1-input"
processFolder = "2-process"
outputFolder = "3-output"
logsFolder = "4-logs"

dataVisualizationTopLimit = 20

testSize = 0.25
randomState = 0
partitionsNumber = 0

In [23]:
# Función para leer datos de un archivo csv
def readData(filePath, delimiter, encoding, header):
  data = pd.read_csv(filePath, delimiter=delimiter,encoding=encoding, header=header)
  return data

In [24]:
def preprocessingData(df):
  # Aca no se realizará ni parseo ni nada, ya vendrá desde la fuente
  # Solo se leerá, se dividirá y se entrenará

  # Balanceo de datos: Sobremuestreo aleatorio (oversampling)
  objectiveColumn = "hired"
  class0, class1 = df[objectiveColumn].value_counts()
  dfClass0 = df[df[objectiveColumn] == 0]
  dfClass1 = df[df[objectiveColumn] == 1]
  dfClass1Sampled = dfClass1.sample(class0, replace=True)
  df = pd.concat([dfClass0, dfClass1Sampled],axis=0)
  #print(df[objectiveColumn].value_counts())
  #display(df)

  # Desordenando la data para evitar sesgos(filas)
  df = df.sample(frac = 1, random_state=randomState).reset_index(drop=True)  
  #print(df[objectiveColumn].value_counts())
  #display(df)

  # Oversampling de la data

  # Aplicando OneHotEncoding a las variables categóricas (transformación a numéricas y normalización)
  categoricalColumns = [columnName for columnName, columnType in df.dtypes.to_dict().items() if columnName not in [ "hired" ] and columnType == "object" ]
  for column in categoricalColumns:
    dummies = pd.get_dummies(df[[column]], prefix=column, drop_first=True, dummy_na=True)
    df = pd.concat([df, dummies], axis = 1)
    df = df.drop(columns=[column])

  # Aplicando MinMaxScaler a las variables numéricas (normalización)
  numericalColumns = [columnName for columnName, columnType in df.dtypes.to_dict().items() if columnName not in [ "hired" ] and columnType == "float64" ]
  for column in numericalColumns:
    df[column] =  df[column].fillna(0.0)

  mms = MinMaxScaler()
  df[numericalColumns] = mms.fit_transform(df[numericalColumns])

  # Lectura de las variables de características y objetivo
  objectiveColumn = "hired"
  X = df.drop([objectiveColumn], axis=1)
  y = df[objectiveColumn]

  return X, y

In [25]:
def splitData(X, y, partitionNumber):
  # Dividiendo los dataframes de entrenamiento y prueba
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = testSize, random_state=randomState)

  if partitionNumber != 0:
    # Obteniendo el total de filas del dataframe de testeo
    X_train, X_test, y_train, y_test = X_train.reset_index(drop=True), X_test.reset_index(drop=True), y_train.reset_index(drop=True), y_test.reset_index(drop=True)
    totalRows = len(X_train)

    # Determinando el límite inferior (primera fila) de la partición
    bottomLimit = int(totalRows*testSize*(partitionNumber-1)) + 1

    # Determinando el límite superior (última fila) de la partición
    topLimit = int(totalRows*testSize*partitionNumber)

    # Determinando una lista con todas los numeros de las filas con la partición
    testList = [x for x in range(bottomLimit-1, topLimit)]

    # Determinando los dataframes de las categorías
    X_train, X_test = X_train[~X_train.index.isin(testList)], X_train[X_train.index.isin(testList)]

    # Determinando los dataframes de los objetivos
    y_train, y_test = y_train[~y_train.index.isin(testList)], y_train[y_train.index.isin(testList)]

  return X_train, X_test, y_train, y_test

In [26]:
def createClassifier(modelName, parameters):
  if modelName == "KNN":
    return KNeighborsClassifier(parameters) if parameters is not None else KNeighborsClassifier()
  elif modelName == "LR":
    return LogisticRegression(parameters) if parameters is not None else LogisticRegression()
  elif modelName == "GNB":
    return GaussianNB(parameters) if parameters is not None else GaussianNB()
  elif modelName == "DT":
    return DecisionTreeClassifier(parameters) if parameters is not None else DecisionTreeClassifier()
  elif modelName == "SVM":
    return SVC(parameters) if parameters is not None else SVC()
  elif modelName == "RF":
    return RandomForestClassifier(parameters) if parameters is not None else RandomForestClassifier()
  elif modelName == "GB":
    return GradientBoostingClassifier(parameters) if parameters is not None else GradientBoostingClassifier()
  else:
    return KNeighborsClassifier()

In [27]:
def trainModel(X_train, X_test, y_train, y_test, modelName, parameters = None):
  # Creación del clasificador KNN
  clf = createClassifier(modelName, parameters)

  # Entrenamiento del clasificador KNN
  clf.fit(X_train, y_train)

  # Calculando la predicción del modelo con la data de prueba
  y_pred = clf.predict(X_test)

  return y_test, y_pred

In [28]:
def getMetrics(y_train, y_test, y_pred, startDate, endDate, partitionNumber, algorithm = "KNN"):
  trainRows = len(y_train)
  testRows = len(y_test)

  # Calculando la exactitud del modelo
  accuracy = "{:.2%}".format(accuracy_score(y_test, y_pred))

  # Calculando la precisión del modelo
  precision = "{:.2%}".format(precision_score(y_test, y_pred))

  # Calculando la sensibilidad del modelo
  recall = "{:.2%}".format(recall_score(y_test, y_pred))

  # Calculando el valor F del modelo (robustez)
  f1Score = "{:.2%}".format(f1_score(y_test, y_pred))

  # Calculando el tiempo de ejecución del modelo
  executionTime = str(int((endDate - startDate).total_seconds() * 1000)) + "ms"

  confussionMatrix = str(confusion_matrix(y_test, y_pred).tolist())
  
  return {
    "algoritmo": algorithm,
    "tipo": "Total de datos" if partitionNumber == 0 else "Particion " + str(partitionNumber),
    "registrosEntrenamiento": trainRows,
    "registrosPrueba": testRows,
    "tiempoEjecucion": executionTime,
    "matrizConfusion": confussionMatrix,
    "exactitud": accuracy,
    "precision": precision,
    "sensibilidad": recall,
    "valorF": f1Score,
  }


In [29]:
def main():
  # Definiendo el inicio del proceso
  startTime = datetime.datetime.now()
  print("Inicio: " + str(startTime))
  print("Se inició el procesamiento")
  print()

  # Leyendo la data
  data = readData(os.path.join(inputFolder, "result.csv"), ',', 'utf-8', 0)

  # Determinando los dataframes de las categorías (X) y el objetivo (y)
  X, y = preprocessingData(data)

  # Creando el arreglo de metricas de cada partición
  metricsList = []

  # Iterando sobre cada partición
  for partitionNumber in range(0,partitionsNumber+1):
    # Separando data para el entrenamiento y testeo
    X_train, X_test, y_train, y_test = splitData(X, y, partitionNumber)

    models = ["KNN", "LR", "GNB", "DT", "RF", "GB"]

    for model in models:
      print("Calculando para el algoritmo {}".format(model))

      # Inicio de ejecución del modelo
      startDate = datetime.datetime.now()
      print("Inicio: " + str(startDate))
      
      # Realizar entrenamiento del modelo
      y_test, y_pred = trainModel(X_train, X_test, y_train, y_test, model)

      # Fin de ejecución del modelo
      endDate = datetime.datetime.now()
      print("Fin: " + str(endDate))
      print("Tiempo: " + str(endDate-startDate))
      print()

      # Obteniendo las métricas de la partición del modelo
      metrics = getMetrics(y_train, y_test, y_pred, startDate, endDate, partitionNumber, model)

      # Añadiendo la métrica de la partición a la lista de métricas
      metricsList.append(metrics)

  # Ordenando las métricas
  metricsList = sorted(metricsList, key=lambda x: (x["algoritmo"], x["tipo"]))

  # Mostrando las métricas
  metricsListPretty = json.dumps(metricsList, indent=4)
  print(metricsListPretty)
    
  # Escribiendo las metricas en un archivo de salida
  with open(os.path.join(outputFolder, datetime.datetime.now().strftime("%Y-%m-%d %H-%M-%S") + ".json"), "w") as f:
      json.dump(metricsList, f, indent=4, ensure_ascii=False)

  # Definiendo el fin del proceso
  endTime = datetime.datetime.now()
  print("Fin: " + str(endTime))
  print("Tiempo: " + str(endTime-startTime))
  
  # Retornando la lista de métricas
  return metricsList

In [30]:
if __name__ == "__main__":
  main()

Inicio: 2023-05-22 11:26:50.904205
Se inició el procesamiento

9722 503
9722
503
0    9722
1    9722
Name: hired, dtype: int64


,jobProfileName,candidateResidenceCountry,candidateCivilStatus,candidateBirthCountry,salary,lastWorkCompany,lastWorkCountry,lastWorkArea,lastWorkName,worksNumber,...,lastEducationCountry,lastEducationArea,lastEducationName,lastEducationStatus,lastEducationDegree,studiesNumber,technicalSkills,languages,otherSkills,hired
0,Cloud Specialist,Peru,Casado/A,Peru,14600.0,Tgestiona Sac,Peru,Infraestructura,Administrador De Servidores,20.0,...,Peru,Computacion / Informatica,Bachiller En Computacion E Informatica,Graduado,Universitario,13.0,35.0,2.2,18.0,0
1,Data Engineer,Peru,Soltero/A,Peru,14600.0,Universidad Privada Del Norte Sac,Peru,Analisis De Datos,Especialista De Gestion De Informacion Y Anali...,20.0,...,Peru,Computacion / Informatica,Maestria En Data Science,En Curso,Master,13.0,35.0,2.2,18.0,0
2,Especialista En Analitica,Peru,Soltero/A,Peru,14600.0,Banco Interamericano De Desarrollo (Bid) - Sunat,Peru,Liderazgo De Proyecto,Consultor Especialista,20.0,...,Peru,Computacion / Informatica,Maestria En Data Science,Graduado,Posgrado,13.0,35.0,2.2,18.0,0
3,Analista De Sistemas,Peru,Casado/A,Peru,14600.0,Stefanini It Solutions,Peru,Programacion,Analista Programador,20.0,...,Peru,Computacion / Informatica,Ingenieria De Sistemas,En Curso,Universitario,13.0,35.0,2.2,18.0,0
4,Gestor De Servicios De Ti,Peru,Casado/A,Peru,14600.0,Navarro &Amp; Ganico,Peru,Otros,Gerente De Proyectos,20.0,...,Peru,Computacion / Informatica,Ingenieria De Sistemas,En Curso,Universitario,13.0,35.0,2.2,18.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1311,Analista De Calidad,Peru,Soltero/A,Peru,14600.0,Canvia S.A.C.,Peru,Testing / Qa / Qc,Practicante/Analista Calidad,20.0,...,Peru,Ing. En Sistemas,Ingenieria De Sistemas,Graduado,Universitario,13.0,35.0,2.2,18.0,1
5014,Community Manger,Peru,Soltero/A,Peru,14600.0,Agencia Kunturi,Peru,Creatividad,Diseno Grafico Y Community Manager,20.0,...,Peru,Comunicacion Social,Ciencias De La Comunicacion,Graduado,Posgrado,13.0,35.0,2.2,18.0,1
8999,Ejecutivo Comercial,Peru,Soltero/A,Peru,14600.0,Bomi Group Peru,Peru,Comercial,Ejecutivo Comercial,20.0,...,Peru,Finanzas,Diplomado En Evaluacion De Proyectos De Inversion,Graduado,Posgrado,13.0,35.0,2.2,18.0,1
6401,Automatizador De Procesos,Peru,Soltero/A,Peru,14600.0,Banco Pichincha,Peru,Ingenieria De Procesos,Analista De Transformacion De Procesos Senior,20.0,...,Peru,Ing. Industrial,Especializacion En Gestion Integral De La Calidad,En Curso,Universitario,13.0,35.0,2.2,18.0,1


1    9722
0    9722
Name: hired, dtype: int64


,jobProfileName,candidateResidenceCountry,candidateCivilStatus,candidateBirthCountry,salary,lastWorkCompany,lastWorkCountry,lastWorkArea,lastWorkName,worksNumber,...,lastEducationCountry,lastEducationArea,lastEducationName,lastEducationStatus,lastEducationDegree,studiesNumber,technicalSkills,languages,otherSkills,hired
0,Cloud Devops,Peru,Casado/A,Peru,14600.0,Banco De Credito Del Peru - Bcp,Peru,Tecnologias De La Informacion,Especialista Devops / Arquitecto Backend,20.0,...,Peru,Ing. En Sistemas,Pre-Grado 8Vo Ciclo,En Curso,Universitario,13.0,35.0,2.2,18.0,1
1,Ejecutivo Comercial,Peru,Soltero/A,Peru,14600.0,Cryptocash,Peru,Desarrollo De Negocios,Asesora De Negocios,20.0,...,Peru,Diseno Grafico,Diseno Grafico,En Curso,Otro,13.0,35.0,2.2,18.0,0
2,Jefe De Proyecto - Analytics & Ai,Peru,Soltero/A,Peru,14600.0,Tecnologia Mco S.A.C.,Peru,Tecnologias De La Informacion,Project Manager,20.0,...,Peru,Otra,Taller De Preparacion Para El Examen Pmp,Graduado,Otro,13.0,35.0,2.2,18.0,1
3,Analista De Base De Datos,Peru,Union Libre,Peru,14600.0,Cia Minera Raura,Peru,Analisis De Datos,Asistente De Base De Datos,20.0,...,Peru,Ing. Informatica,Especialista En Base De Datos,En Curso,Posgrado,13.0,35.0,2.2,18.0,0
4,Analista De Sistemas,Peru,Casado/A,Peru,14600.0,Check Y Check,Peru,Sistemas,Jefe De Sistemas,20.0,...,Peru,Ing. En Sistemas,Abap,Graduado,Master,13.0,35.0,2.2,18.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19439,Analista De Sistemas,Peru,Soltero/A,Peru,14600.0,America Movil Peru S.A.C.,Peru,Liderazgo De Proyecto,Analista Sr. Ti,20.0,...,Peru,Telecomunicaciones,Cbio 16 Catalog Manager And Order Care Advanced,Graduado,Otro,13.0,35.0,2.2,18.0,0
19440,Community Manger,Peru,Soltero/A,Peru,14600.0,Worked Music,Peru,Community Management,Community Manager,20.0,...,Peru,Marketing / Comercializacion,Especializacion En Facebook Y Google Ads,Graduado,Otro,13.0,35.0,2.2,18.0,1
19441,Desarrollador .Net,Peru,Soltero/A,Peru,14600.0,Ministerio De La Produccion,Peru,Testing / Qa / Qc,Analyst Quality Assurance,20.0,...,Peru,Analisis De Sistemas,Calidad Y Gestion De Software,En Curso,Otro,13.0,35.0,2.2,18.0,1
19442,Jefe De Proyecto - Modern Workplace,Peru,Soltero/A,Peru,14600.0,Crosland Sac,Peru,Tecnologia / Sistemas,Analista Senior,20.0,...,Peru,Ing. En Sistemas,Maestria Ingenieria De Sistemas,En Curso,Master,13.0,35.0,2.2,18.0,1


Calculando para el algoritmo KNN
Inicio: 2023-05-22 11:26:54.390533
Fin: 2023-05-22 11:27:03.336932
Tiempo: 0:00:08.946399

Calculando para el algoritmo LR
Inicio: 2023-05-22 11:27:03.342909
